# Week 3 - Sensitivity analysis

This exercise uses the same predator-prey model we used for the multi-model exercise, focusing on the Python version. As with the other exercise, define a model object for the function below, with the uncertainty ranges provided:

|Parameter	|Range or value	        |
|-----------|--------------:|
|prey_birth_rate    	|0.015 – 0.035	|
|predation_rate|0.0005 – 0.003 	|
|predator_efficiency     	|0.001 – 0.004	    |
|predator_loss_rate	    |0.04 – 0.08	    |

In [5]:
import numpy as np
import matplotlib.pyplot as plt

from ema_workbench import Model, RealParameter, TimeSeriesOutcome, perform_experiments, ema_logging

from ema_workbench import Samplers

from ema_workbench.analysis import feature_scoring
from ema_workbench.analysis.scenario_discovery_util import RuleInductionType
from ema_workbench.em_framework.salib_samplers import get_SALib_problem
from SALib.analyze import sobol

In [2]:
def pred_prey(prey_birth_rate=0.025, predation_rate=0.0015, predator_efficiency=0.002,
             predator_loss_rate=0.06, initial_prey=50, initial_predators=20, dt=0.25,
             final_time=365, reps=1):

    #Initial values
    predators = np.zeros((reps, int(final_time/dt)+1))
    prey = np.zeros((reps, int(final_time/dt)+1))
    sim_time = np.zeros((reps, int(final_time/dt)+1))

    for r in range(reps):

        predators[r,0] = initial_predators
        prey[r,0] = initial_prey

    #Calculate the time series
    for t in range(0, sim_time.shape[1]-1):

        dx = (prey_birth_rate*prey[r,t]) - (predation_rate*prey[r,t]*predators[r,t])
        dy = (predator_efficiency*predators[r,t]*prey[r,t]) - (predator_loss_rate*predators[r,t])

        prey[r,t+1] = max(prey[r,t] + dx*dt, 0)
        predators[r,t+1] = max(predators[r,t] + dy*dt, 0)
        sim_time[r,t+1] = (t+1)*dt

    #Return outcomes
    return {'TIME':sim_time,
            'predators':predators,
            'prey':prey}

In [7]:
from ema_workbench import Model, RealParameter, TimeSeriesOutcome # Constant #TimeSeriesOutcome #IntegerParameter, BinaryParameter, CategoricalParameter

model = Model('PredPrey', function=pred_prey)

# Specify uncertainties, names of parameters need to match key word in the function
model.uncertainties = [RealParameter('prey_birth_rate', 0.015, 0.035), #IntegerParameter, BinaryParameter, CategoricalParameter
                       RealParameter('predation_rate', 0.0005, 0.003),
                       RealParameter('predator_efficiency',0.001, 0.004),
                       RealParameter('predator_loss_rate',0.04, 0.08)]

# Set levers, one for each time step
#model.levers = [RealParameter('l'+str(i), 0, 0.1) for i in range(100)]

# Specify outcomes
model.outcomes = [TimeSeriesOutcome('TIME'), #time
                  TimeSeriesOutcome('predators'), #predator
                  TimeSeriesOutcome('prey')] #prey

#model.constants = [Constant('final_time', 365), 
#                    Constant('dt', 0.25)]

#### 1. Sensitivity analysis
Sensitivity analysis often focuses on the final values of an outcome at the end of the simulation. However, we can also look at metrics that give us additional information about the behavior of the model over time. Using [the statsmodel library](https://www.statsmodels.org/stable/index.html) and an appropriate sampling design, fit a linear regression model for each of the following indicators. What can we conclude about the behavior of the model, and about the importance of the different inputs?

  * The final values of the _prey_ outcome
  * The mean values of the _prey_ outcome over time, within each experiment
  * The standard deviations of the _prey_ outcome over time, within each experiment

In [8]:
import statsmodels.api as sm

import statsmodels.formula.api as smf

In [9]:
#Sampling design
from ema_workbench import SequentialEvaluator

with SequentialEvaluator(model) as evaluator:
    experiments, outcomes = evaluator.perform_experiments(scenarios=50) #default sampling method uniform distribution with Latin-Hypercube


100%|█████████████████████████████████████████| 50/50 [00:00<00:00, 129.90it/s]


In [10]:
experiments

,predation_rate,predator_efficiency,predator_loss_rate,prey_birth_rate,scenario,policy,model
0,0.001200,0.002085,0.070508,0.018696,0,None,PredPrey
1,0.002262,0.003287,0.052081,0.021858,1,None,PredPrey
2,0.002465,0.002920,0.046979,0.016605,2,None,PredPrey
3,0.001650,0.002265,0.041849,0.028855,3,None,PredPrey
4,0.002659,0.002538,0.075270,0.023005,4,None,PredPrey
5,0.001823,0.001741,0.057729,0.019374,5,None,PredPrey
6,0.000788,0.003523,0.073030,0.016537,6,None,PredPrey
7,0.002158,0.001405,0.078857,0.023816,7,None,PredPrey
8,0.002708,0.001991,0.060624,0.017331,8,None,PredPrey
9,0.001558,0.001070,0.066570,0.027557,9,None,PredPrey


In [50]:
lastout = []
for i in range(0,50,1):
    lastout.append(outcomes['prey'][i][0][1460]) #a time series of possible outcomes at 
lastout

[49.49308686574551,
 3.5913245254361414,
 2.7957825671489496,
 17.909758539276293,
 14.909126231812124,
 52.565616195489284,
 13.320945054467883,
 50.22135675048127,
 26.38998896835544,
 72.89789818616244,
 42.24085469456313,
 14.276836880358038,
 16.054901534951448,
 51.45345530455229,
 46.37370967149409,
 12.88159626267565,
 3.1619651061490184,
 45.09686113875378,
 13.647185621654785,
 32.19867090306169,
 29.090882817439695,
 38.20211829948692,
 17.308833900991697,
 2.6480962997861153,
 61.085080433071624,
 0.4844463954589471,
 53.08991388541312,
 60.31376179519413,
 18.939762988460505,
 55.23269396630813,
 45.91685595896273,
 3.5360653926582684,
 7.576779032614557,
 25.317902239584765,
 6.985581495791229,
 45.53363388867848,
 23.76783959610862,
 54.1119696055869,
 56.58207529838989,
 54.60217448729153,
 8.177538454046433,
 8.64202878052567,
 16.056823117500127,
 9.207780667023128,
 21.011469100400785,
 0.5079291163959797,
 51.426883724908734,
 8.490300023501671,
 65.35824079207359,


In [61]:
#Linear regression for prey outcome
model = sm.OLS(experiments['predation_rate'].values, np.array(lastout))
results1 = model.fit()
print(results.summary())

model = sm.OLS(experiments['predator_efficiency'].values, np.array(lastout))
results2 = model.fit()
print(results2.summary())

model = sm.OLS(experiments['predator_loss_rate'].values, np.array(lastout))
results3 = model.fit()
print(results3.summary())

model = sm.OLS(experiments['prey_birth_rate'].values, np.array(lastout))
results4 = model.fit()
print(results4.summary())

#Linear regression for mean prey outcome over time 
lastoutm = []
for i in range(0,50,1):
    lastoutm.append(outcomes['prey'][i][0].mean()) #a time series of possible outcomes at 
lastoutm

model = sm.OLS(experiments['predation_rate'].values, np.array(lastoutm))
results5 = model.fit()
print(results5.summary())

#Linear regression for sdv prey outcome over time
lastouts = []
for i in range(0,50,1):
    lastouts.append(outcomes['prey'][i][0].std()) #a time series of possible outcomes at 
lastouts

model = sm.OLS(experiments['predation_rate'].values, np.array(lastouts))
results6 = model.fit()
print(results6.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.535
Model:                            OLS   Adj. R-squared (uncentered):              0.526
Method:                 Least Squares   F-statistic:                              56.39
Date:                Thu, 11 May 2023   Prob (F-statistic):                    1.08e-09
Time:                        17:04:48   Log-Likelihood:                          261.73
No. Observations:                  50   AIC:                                     -521.5
Df Residuals:                      49   BIC:                                     -519.5
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

#### 2. SOBOL
Use the Sobol sampling functionality included in the Workbench to perform experiments with a sample size of N=50, then analyze the results with SALib for the same three indicators. This requires specifying the keyword argument `'uncertainty_sampling'` of perform_experiments. Note that when using Sobol sampling, the meaning of the keyword argument `scenarios` changes a bit. In order to properly estimate Sobol scores as well as interaction effects, you require N * (2D+2) scenarios, where D is the number of uncertain parameters, and N is the value for scenarios passed to `perform_experiments`. Repeat the analysis for larger sample sizes, with N=250 and N=1000. How can we interpret the first-order and total indices? Are these sample sizes sufficient for a stable estimation of the indices? You'll need to use the [get_SALib_problem](https://emaworkbench.readthedocs.io/en/latest/ema_documentation/em_framework/salib_samplers.html) function to convert your Workbench experiments to a problem definition that you can pass to the SALib analysis function.

*hint*: sobol is a deterministic sequence of quasi random numbers. Thus, you can run with N=1000 and simply use slicing to get the results for N=50 and N=250.

#### 3. Extra trees
Use the [Extra-Trees analysis](https://emaworkbench.readthedocs.io/en/latest/ema_documentation/analysis/feature_scoring.html) included in the Workbench to approximate the Sobol total indices, with a suitable sampling design. As a starting point, use an ensemble of 100 trees and a max_features parameter of 0.6, and set the analysis to regression mode. Are the estimated importances stable relative to the sample size and the analysis parameters? How do the results compare to the Sobol indices? For more details on this analysis see [Jaxa-Rozen & Kwakkel (2018)](https://www.sciencedirect.com/science/article/pii/S1364815217311581)